In [70]:
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup as bs
from random import shuffle
from re import sub

In [71]:
def get_soup(url):
    '''url을 받아 소스코드를 긁어온 후 
    BeautifulSoup로 파싱한 것을 반환합니다.
    '''
    with request.urlopen(url) as f:
        html = f.read().decode('utf-8')
    soup = bs(html, 'html5lib')
    return soup

In [79]:
def get_pets(soup):
    '''BeautifulSoup로 파싱된 소스코드를 받아 
    해당 페이지의 하단 청원목록 항목들을 리스트로 반환합니다
    '''
    pet_body = soup.select("div.bl_body ul")
    pet_list = pet_body[1]
    pet_items = pet_list.select("li div.bl_wrap")
    return pet_items

In [80]:
def get_text(elem, choice):
    '''청원목록 항목 내의 태그(elem)와 노드 위치(choice)를 받아
    태그의 텍스트 노드를 찾아 텍스트를 반환합니다. 
    '''
    elem_children = list(elem.children)
    elem_text = elem_children[choice]
    elem_trim = elem_text.strip()
    return elem_trim

In [81]:
def get_item_info(item, choice=[1, 1, 1, 2]):
    '''청원목록의 한 항목을 받아 
    항목의 번호, 분류, 제목, 참여인원을 리스트로 반환합니다.
    잘못된 텍스트 노드가 반환될 경우 choice를 0|1|2로 직접 설정해보시기 바랍니다. 
    '''
    num_choice, category_choice, title_choice, votes_choice = choice
    num_raw = item.select("div.bl_no")[0]
    category_raw = item.select("div.bl_category")[0]
    title_raw = item.select("div.bl_subject a")[0]
    votes_raw = item.select("div.bl_agree")[0]
    
    item_info = [get_text(num_raw, num_choice), get_text(category_raw, category_choice), 
                 get_text(title_raw, title_choice), get_text(votes_raw, votes_choice)]
    return item_info


In [82]:
def crawl_page(url):
    '''청원목록 페이지 url을 받아 
    페이지의 항목(15개)를 추출해 데이터프레임 형태로 반환합니다.
    '''
    soup = get_soup(url)
    pet_items = get_pets(soup)
    lst = []
    for i in pet_items:
        lst.append(get_item_info(i))
    df = pd.DataFrame(lst, columns=['번호', '분류', '제목', '참여인원'])
    df['참여인원'] = df['참여인원'].str.replace('명', '').astype("int64") 
    return df

In [83]:
def get_top_id():
    '''가장 최근에 등록된 청원의 번호(id)를 int로 반환합니다
    '''
    url = "https://www1.president.go.kr/petitions"
    soup = get_soup(url)
    pet_top = get_pets(soup)[0]
    pet_top_id = int(get_item_info(pet_top)[0])
    return pet_top_id

In [84]:
def crawl_pet(petnum):
    '''청원글 본문 번호를 받아 해당 청원글의 제목과 본문을 추출,
    리스트로 반환.
    '''
    url = "https://www1.president.go.kr/petitions/" + str(petnum)
    soup = get_soup(url)
    title = soup.find("h3", class_="petitionsView_title").text
    content_raw = soup.find("div", class_="View_write").text
    content = sub("\n\t+", '', content_raw)
    return ([title, content])

# 기본과제

국민청원 첫 페이지 하단의 **청원 목록**에서 **번호, 분류, 제목, 참여인원**을 추출하여 **DataFrame**에 담아주세요.

In [85]:
url = "https://www1.president.go.kr/petitions"
df = crawl_page(url)
df 

,번호,분류,제목,참여인원
0,242059,미래,대한민국 군대 선택제 청원합니다.,0
1,242058,일자리,"현대판 노예법 근로기준법63조(감,단직 근로자 승인) ...",0
2,242057,기타,노숙자분들이 쉴수있는 공간을 만들어주세요,1
3,242056,기타,국민연금=공무원연금통합시켜라 국민세금 도둑질하지 ...,0
4,242055,기타,위조신분증으로 술먹고 돈받으면 신고한다는 미성년자 ...,0
5,242054,기타,칭찬해주세요,0
6,242053,안전/환경,아침에 밥좀 먹고 나오자 건설현장 출근 시간을 아침 ...,0
7,242052,정치개혁,"공무원 정년(50세)축소,연금폐지!!!",1
8,242051,기타,문재앙씨 나라 어디까지 말아먹을건가?,1
9,242050,안전/환경,나무(숲) 가꾸기 관련,0


# 선택과제 1

국민청원 목록 첫 페이지 주소는 https://www1.president.go.kr/petitions?page=1 입니다. 두번째 페이지 주소는 https://www1.president.go.kr/petitions?page=2 입니다. **처음 10개 페이지**에 순차적으로 방문하여 **번호, 분류, 제목, 참여인원**을 추출하고 그 결과를 **DataFrame**에 담아주세요.

In [88]:
url_base = "https://www1.president.go.kr/petitions?page="
df = pd.DataFrame([], columns=['번호', '분류', '제목', '참여인원'])
for i in range(1,11):
    url_temp = url_base + str(i)
    df_temp = crawl_page(url_temp)
    df = df.append(df_temp, ignore_index=True)
    print("페이지 %d를 크롤링 중입니다"%i)
    
print("크롤링 완료.")    

페이지 1를 크롤룅 중입니다
페이지 2를 크롤룅 중입니다
페이지 3를 크롤룅 중입니다
페이지 4를 크롤룅 중입니다
페이지 5를 크롤룅 중입니다
페이지 6를 크롤룅 중입니다
페이지 7를 크롤룅 중입니다
페이지 8를 크롤룅 중입니다
페이지 9를 크롤룅 중입니다
페이지 10를 크롤룅 중입니다
크롤링 완료.


In [90]:
print(df.shape)
df.sample(15)

(150, 4)


,번호,분류,제목,참여인원
9,242052,정치개혁,"공무원 정년(50세)축소,연금폐지!!!",1
32,242029,정치개혁,LET''S ELIMINATE THE SPECTER OF MILITARY DICTA...,0
56,242005,기타,환경미화원 근무복 개선 요청,1
101,241960,문화/예술/체육/언론,자전거 탈때 헬맷사용?,0
92,241969,문화/예술/체육/언론,(주) 조이시티 모바일게임인 주사위의신의 이벤트 확 ...,2
80,241981,인권/성평등,갑질하는 KT를 소송해습니다.,0
8,242053,안전/환경,아침에 밥좀 먹고 나오자 건설현장 출근 시간을 아침 ...,0
62,241999,육아/교육,모순의 극치인 학생부종합전형 폐지를 청원합니다.,3
55,242006,성장동력,대통령은 잠이 오십니까? 20만명이 안되도 이런 주제 ...,12
107,241954,일자리,경행특채 처우 대폭 개선,1


# 선택과제 2

2018년 7월 17일 현재 국민청원의 글 번호는 1번부터 238663번까지 부여되어 있습니다. 하지만 중간중간 *삭제된 청원*이 있어서 실제 청원 수는 238663개보다 적습니다. 309510번 청원글 본문의 URL은 https://www1.president.go.kr/petitions/309510 입니다.
URL에서 글번호만 바꾸면 해당 청원의 본문으로 이동할 수 있습니다. **이러한 URL 패턴의 특성을 활용**하여 전체 청원 중 **임의로 100개** 청원의 **제목과 본문**을 추출하여 **DataFrame**에 담아주세요.

In [94]:
url_base = "https://www1.president.go.kr/petitions/"
topid = get_top_id()
pool = list(range(1, topid+1))
picked = []
article_lst = []

while len(picked) < 100:
    shuffle(pool)
    pet = pool.pop()
    try:
        article = crawl_pet(pet)
        article_lst.append(article)
        picked.append(pet)
    except:
        print('삭제된 청원입니다. 글 번호: %d, 총 크롤링 수: %d'%(pet, len(article_lst)) )
    
df = pd.DataFrame(article_lst, columns=["제목", "본문"])
print("청원 크롤링이 완료되었습니다. 총 크롤링 수: %d" %len(df))

삭제된 청원입니다. 글 번호: 217812, 총 크롤링 수: 3
삭제된 청원입니다. 글 번호: 107663, 총 크롤링 수: 6
삭제된 청원입니다. 글 번호: 114312, 총 크롤링 수: 6
삭제된 청원입니다. 글 번호: 104243, 총 크롤링 수: 8
삭제된 청원입니다. 글 번호: 163406, 총 크롤링 수: 8
삭제된 청원입니다. 글 번호: 108573, 총 크롤링 수: 25
삭제된 청원입니다. 글 번호: 222337, 총 크롤링 수: 36
삭제된 청원입니다. 글 번호: 216170, 총 크롤링 수: 36
삭제된 청원입니다. 글 번호: 81312, 총 크롤링 수: 43
삭제된 청원입니다. 글 번호: 107742, 총 크롤링 수: 47
삭제된 청원입니다. 글 번호: 117180, 총 크롤링 수: 48
삭제된 청원입니다. 글 번호: 226359, 총 크롤링 수: 48
삭제된 청원입니다. 글 번호: 97974, 총 크롤링 수: 56
삭제된 청원입니다. 글 번호: 99208, 총 크롤링 수: 57
삭제된 청원입니다. 글 번호: 95565, 총 크롤링 수: 57
삭제된 청원입니다. 글 번호: 238842, 총 크롤링 수: 69
삭제된 청원입니다. 글 번호: 86339, 총 크롤링 수: 73
삭제된 청원입니다. 글 번호: 216782, 총 크롤링 수: 80
삭제된 청원입니다. 글 번호: 218012, 총 크롤링 수: 81
삭제된 청원입니다. 글 번호: 90692, 총 크롤링 수: 96
삭제된 청원입니다. 글 번호: 143417, 총 크롤링 수: 96
청원 크롤링이 완료되었습니다. 총 크롤링 수: 100


In [95]:
print(df.shape)
df.sample(10)

(100, 2)


,제목,본문
32,육아휴직 실태조사 해주세요,육아휴직 제도만 되어있으면 뭐하나요 남자들은 쓰지도 못하고 여자들은 3개월 또는...
18,학교생활기록부 간소화 정책추진을 멈춰주세요,02년생들은 문이과 통합으로 고심하고 또 고심해서 고등학교 입시를 끝내고 입학을 한...
98,정현식 판사의 파면 청원을 한곳에 모아야 별을 따지요..!!,청원 내용이 너무 분산되어 있습니다..분산시키면 절대 청원에 대한 결과를 볼수 없으...
54,광복절날만 애국타령하는 국민과 정치인 그리고 기레기언론인,위안부나 징용자에 대해선 침이 튀기고 피가 튀는 나라입니다. 그러나 항일애국지사와 ...
27,국회의원 자격조건에 군필과 전과기록이,국회의원 자격조건에 군대와 전과기록에 관하여 조치되었으면 좋겠습니다. 국회의원은 ...
1,조두순무기징역,조두순을 재심사해 무기징역으로 해주세요
3,국회의원도 정년퇴직 나이를 정해주세요.,국회의원은 국민을 대표하는 자리이므로 할 일이 정말 많다고 생각합니다. 공무원이나 ...
44,나경원 평창올림픽 위원자격을 박탈해주시도록 청원합니다..,평화올림픽이 되도록 함께 온국민과 함께 마음을 모아야 할분이.. 분열일으킬 말이나 ...
51,우파적페청산후 좌파 청산 부탁드립니다.,지금은 수구 우파들 청산이 급하니 참고있습니다. 어드정도 정리되면 좌파 적폐도 해...
91,갑질 판결 현실,"결탁의 엉터리 판결을 주장한다면, 들어시겠습니까? 2014카합103호와 2015가단..."
